In [1]:
#导入所需要的包和模块
import collections#collections是Python内建的一个集合模块，提供了许多有用的集合类
import gluonbook as gb
import math
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import data as gdata, loss as gloss, model_zoo, nn
import os
import shutil
import time
import zipfile
data_dir = './zjc'

In [2]:
#从整理数据集开始，reorg_train_valid函数从完整原始训练集中切分出验证集
def reorg_train_valid(data_dir, train_dir, input_dir, valid_ratio, idx_label):
    # 训练集中数量最少一类的狗的样本数。
    min_n_train_per_label = (
        collections.Counter(idx_label.values()).most_common()[:-2:-1][0][1])
    # 验证集中每类狗的样本数。
    n_valid_per_label = math.floor(min_n_train_per_label * valid_ratio)
    label_count = {}
    for train_file in os.listdir(os.path.join(data_dir, train_dir)):
        idx = train_file.split('.')[0]
        label = idx_label[idx]
        gb.mkdir_if_not_exist([data_dir, input_dir, 'train_valid', label])
        shutil.copy(os.path.join(data_dir, train_dir, train_file),
                    os.path.join(data_dir, input_dir, 'train_valid', label))
        if label not in label_count or label_count[label] < n_valid_per_label:
            gb.mkdir_if_not_exist([data_dir, input_dir, 'valid', label])
            shutil.copy(os.path.join(data_dir, train_dir, train_file),
                        os.path.join(data_dir, input_dir, 'valid', label))
            label_count[label] = label_count.get(label, 0) + 1
        else:
            gb.mkdir_if_not_exist([data_dir, input_dir, 'train', label])
            shutil.copy(os.path.join(data_dir, train_dir, train_file),
                        os.path.join(data_dir, input_dir, 'train', label))

In [3]:
def reorg_dog_data(data_dir, label_file, train_dir, test_dir, input_dir,
                   valid_ratio):
    # 读取训练数据标签。
    with open(os.path.join(data_dir, label_file), 'r') as f:
        # 跳过文件头行（栏名称）。
        lines = f.readlines()[1:]
        tokens = [l.rstrip().split(',') for l in lines]
        idx_label = dict(((idx, label) for idx, label in tokens))
    reorg_train_valid(data_dir, train_dir, input_dir, valid_ratio, idx_label)
    # 整理测试集。
    gb.mkdir_if_not_exist([data_dir, input_dir, 'test', 'unknown'])
    for test_file in os.listdir(os.path.join(data_dir, test_dir)):
        shutil.copy(os.path.join(data_dir, test_dir, test_file),
                    os.path.join(data_dir, input_dir, 'test', 'unknown'))

In [4]:
if False:#demo:
    # 注意：此处使用小数据集并将批量大小相应设小。使用 Kaggle 比赛的完整数据集时可设批量大
    # 小为较大整数。
    input_dir, batch_size = '', 1
else:
    label_file, train_dir, test_dir = 'labels.csv', 'train', 'test'
    input_dir, batch_size, valid_ratio = 'train_valid_test', 128, 0.1
    reorg_dog_data(data_dir, label_file, train_dir, test_dir, input_dir,
                   valid_ratio)

In [5]:
#训练集的数据增强
#对图像进行不同方式的裁剪、调整亮度、色彩等。
transform_train = gdata.vision.transforms.Compose([
    # 随机对图像裁剪出面积为原图像面积 0.08 到 1 倍之间、且高和宽之比在 3/4 和 4/3 之间
    # 的图像，再放缩为高和宽均为 224 像素的新图像。

    gdata.vision.transforms.RandomResizedCrop(224, scale=(0.08, 1.0),
                                              ratio=(3.0/4.0, 4.0/3.0)),
    gdata.vision.transforms.RandomFlipLeftRight(),
    # 随机变化亮度、对比度和饱和度。
    gdata.vision.transforms.RandomColorJitter(brightness=0.4, contrast=0.4,
                                              saturation=0.4),
    # 随机加噪音。
    gdata.vision.transforms.RandomLighting(0.1),
    gdata.vision.transforms.ToTensor(),
    # 对图像的每个通道做标准化。
    gdata.vision.transforms.Normalize([0.485, 0.456, 0.406],
                                      [0.229, 0.224, 0.225])])

In [ ]:
#训练集的数据增强
#图像增强：对训练图像做一系列随机改变，来产生相似但又不同的训练样本，从而扩大训练数据集的规模。
#可以降低模型对某些属性的依赖，从而提高模型的泛化能力。
#例如，对图像进行不同方式的裁剪，使感兴趣的物体出现在不同位置，从而让模型减轻对物体出现位置的依赖性。还有调整亮度、色彩等。
transform_train = gdata.vision.transforms.Compose([
    # 随机对图像裁剪出面积为原图像面积 0.08 到 1 倍之间、且高和宽之比在 3/4 和 4/3 之间
    # 的图像，再放缩为高和宽均为 224 像素的新图像。

    gdata.vision.transforms.RandomResizedCrop(224, scale=(0.08, 1.0),
                                              ratio=(3.0/4.0, 4.0/3.0)),
    gdata.vision.transforms.RandomFlipLeftRight(),
    # 随机变化亮度、对比度和饱和度。
    gdata.vision.transforms.RandomColorJitter(brightness=0.4, contrast=0.4,
                                              saturation=0.4),
    # 随机加噪音。
    gdata.vision.transforms.RandomLighting(0.1),
    gdata.vision.transforms.ToTensor(),
    # 对图像的每个通道做标准化。
    gdata.vision.transforms.Normalize([0.485, 0.456, 0.406],
                                      [0.229, 0.224, 0.225])])

In [6]:
#测试集的数据增强
transform_test = gdata.vision.transforms.Compose([
    gdata.vision.transforms.Resize(256),
    # 将图像中央的高和宽均为 224 的正方形区域裁剪出来。
    gdata.vision.transforms.CenterCrop(224),
    gdata.vision.transforms.ToTensor(),
    gdata.vision.transforms.Normalize([0.485, 0.456, 0.406],
                                      [0.229, 0.224, 0.225])])

In [7]:
#创建ImageFolderDataset实例来读取整理后的含原始图像文件的数据集
train_ds = gdata.vision.ImageFolderDataset(
    os.path.join(data_dir, input_dir, 'train'), flag=1)
valid_ds = gdata.vision.ImageFolderDataset(
    os.path.join(data_dir, input_dir, 'valid'), flag=1)
train_valid_ds = gdata.vision.ImageFolderDataset(
    os.path.join(data_dir, input_dir, 'train_valid'), flag=1)
test_ds = gdata.vision.ImageFolderDataset(
    os.path.join(data_dir, input_dir, 'test'), flag=1)

In [8]:
#创建DataLoader实例
train_data = gdata.DataLoader(train_ds.transform_first(transform_train),
                              batch_size, shuffle=True, last_batch='keep')
valid_data = gdata.DataLoader(valid_ds.transform_first(transform_test),
                              batch_size, shuffle=True, last_batch='keep')
train_valid_data = gdata.DataLoader(train_valid_ds.transform_first(
    transform_train), batch_size, shuffle=True, last_batch='keep')
test_data = gdata.DataLoader(test_ds.transform_first(transform_test),
                             batch_size, shuffle=False, last_batch='keep')

In [16]:
#定义模型：利用Gluon 的预训练模型，在这里使用了预训练的 ResNet152_v2 模型
def get_net(ctx):
    finetune_net = model_zoo.vision.resnet50_v2(pretrained=True)  #这里可以选择网络
    # 定义新的输出网络。
    finetune_net.output_new = nn.HybridSequential(prefix='')
    finetune_net.output_new.add(nn.Dense(256, activation='relu'))
    # 120 是输出的类别数。
    finetune_net.output_new.add(nn.Dense(120))
    # 初始化输出网络。
    finetune_net.output_new.initialize(init.Xavier(), ctx=ctx)
    # 把模型参数分配到即将用于计算的 CPU 或 GPU 上。
    finetune_net.collect_params().reset_ctx(ctx)
    return finetune_net

In [17]:
loss = gloss.SoftmaxCrossEntropyLoss()

def get_loss(data, net, ctx):
    l = 0.0
    for X, y in data:
        y = y.as_in_context(ctx)
        output_features = net.features(X.as_in_context(ctx))
        outputs = net.output_new(output_features)
        l += loss(outputs, y).mean().asscalar()
    return l / len(data)

In [19]:
def train(net, train_data, valid_data, num_epochs, lr, wd, ctx, lr_period,
          lr_decay):
    # 只训练我们定义的小规模输出网络。
    trainer = gluon.Trainer(net.output_new.collect_params(), 'sgd',
                            {'learning_rate': lr, 'momentum': 0.9, 'wd': wd})
    #trainer = gluon.Trainer(net.output_new.collect_params(), 'adam',
    #                        {'learning_rate': lr})
    switch_delr = 5
    for epoch in range(num_epochs):
        train_l, start = 0.0, time.time()
        if epoch > 0 and switch_delr == 0:#epoch % lr_period == 0:
            trainer.set_learning_rate(trainer.learning_rate * lr_decay)
        for X, y in train_data:
            y = y.astype('float32').as_in_context(ctx)
            output_features = net.features(X.as_in_context(ctx))
            with autograd.record():
                outputs = net.output_new(output_features)
                l = loss(outputs, y)
            l.backward()
            trainer.step(batch_size)
            train_l += l.mean().asscalar()
        time_s = "time %.2f sec" % (time.time() - start)
        if valid_data is not None:
            global min_valloss,best_epoch
            valid_loss = get_loss(valid_data, net, ctx)
            if valid_loss < min_valloss:
                min_valloss = valid_loss
                best_epoch = epoch + 1
                net.save_parameters('resnet50-best.parameters')
                switch_delr = 5
            else:
                switch_delr -= 1
            epoch_s = ("epoch %d, train loss %f, valid loss %f, "
                       % (epoch + 1, train_l / len(train_data), valid_loss))
        else:
            epoch_s = ("epoch %d, train loss %f, "
                       % (epoch + 1, train_l / len(train_data)))
        print(epoch_s + time_s + ', lr ' + str(trainer.learning_rate))

In [ ]:
#训练并验证模型
import mxnet as mx
min_valloss = 9999
best_epoch = 0
ctx, num_epochs, lr, wd = mx.gpu(7), 100, 0.01, 1e-4 #gpu使用7
lr_period, lr_decay, net = 8, 0.1, get_net(ctx)
net.hybridize()
train(net, train_data, valid_data, num_epochs, lr, wd, ctx, lr_period,
      lr_decay)

epoch 1, train loss 3.421688, valid loss 1.402047, time 79.20 sec, lr 0.01
epoch 2, train loss 1.264278, valid loss 0.661270, time 76.72 sec, lr 0.01
epoch 3, train loss 0.944871, valid loss 0.518590, time 79.47 sec, lr 0.01
epoch 4, train loss 0.853375, valid loss 0.445683, time 78.89 sec, lr 0.01
epoch 5, train loss 0.816127, valid loss 0.437845, time 78.80 sec, lr 0.01
epoch 6, train loss 0.775019, valid loss 0.402784, time 78.71 sec, lr 0.01
epoch 7, train loss 0.761728, valid loss 0.402506, time 79.80 sec, lr 0.01
epoch 8, train loss 0.730868, valid loss 0.401441, time 78.30 sec, lr 0.01
epoch 9, train loss 0.723021, valid loss 0.392653, time 79.15 sec, lr 0.01
epoch 10, train loss 0.697139, valid loss 0.376695, time 78.75 sec, lr 0.01
epoch 11, train loss 0.709589, valid loss 0.394510, time 78.29 sec, lr 0.01
epoch 12, train loss 0.667467, valid loss 0.352477, time 78.66 sec, lr 0.01
epoch 13, train loss 0.667456, valid loss 0.369477, time 78.61 sec, lr 0.01
epoch 14, train loss 

In [21]:
print(min_valloss,best_epoch)

0.3052832509080569 93


In [22]:
#对测试集分类
#net = get_net(ctx)
#net.hybridize()
#train(net, train_valid_data, None, num_epochs, lr, wd, ctx, lr_period,
#      lr_decay)
net.load_parameters('resnet50-best.parameters')
preds = []
for data, label in test_data:
    output_features = net.features(data.as_in_context(ctx))
    output = nd.softmax(net.output_new(output_features))
    preds.extend(output.asnumpy())
ids = sorted(os.listdir(os.path.join(data_dir, input_dir, 'test/unknown')))
with open('submission-resnet50-loss0.30-threshold.csv', 'w') as f:
    f.write('id,' + ','.join(train_valid_ds.synsets) + '\n')
    for i, output in zip(ids, preds):
        f.write(i.split('.')[0] + ',' + ','.join(
            [str(num) for num in output]) + '\n')

In [24]:
#继承学习
#定义模型：利用Gluon 的预训练模型，在这里使用了预训练的 ResNet152_v2 模型
def get_net1(ctx):
    finetune_net = model_zoo.vision.resnet50_v2(pretrained=True)  #这里可以选择网络
    # 定义新的输出网络。
    finetune_net.output_new = nn.HybridSequential(prefix='')
    finetune_net.output_new.add(nn.Dense(256, activation='relu'))
    # 120 是输出的类别数。
    finetune_net.output_new.add(nn.Dense(120))
    # 初始化输出网络。
    finetune_net.output_new.initialize(init.Xavier(), ctx=ctx)
    # 把模型参数分配到即将用于计算的 CPU 或 GPU 上。
    finetune_net.collect_params().reset_ctx(ctx)
    return finetune_net
#定义模型：利用Gluon 的预训练模型，在这里使用了预训练的 ResNet152_v2 模型
def get_net2(ctx):
    finetune_net = model_zoo.vision.resnet101_v2(pretrained=True)  #这里可以选择网络
    # 定义新的输出网络。
    finetune_net.output_new = nn.HybridSequential(prefix='')
    finetune_net.output_new.add(nn.Dense(256, activation='relu'))
    # 120 是输出的类别数。
    finetune_net.output_new.add(nn.Dense(120))
    # 初始化输出网络。
    finetune_net.output_new.initialize(init.Xavier(), ctx=ctx)
    # 把模型参数分配到即将用于计算的 CPU 或 GPU 上。
    finetune_net.collect_params().reset_ctx(ctx)
    return finetune_net
#定义模型：利用Gluon 的预训练模型，在这里使用了预训练的 ResNet152_v2 模型
def get_net3(ctx):
    finetune_net = model_zoo.vision.resnet152_v2(pretrained=True)  #这里可以选择网络
    # 定义新的输出网络。
    finetune_net.output_new = nn.HybridSequential(prefix='')
    finetune_net.output_new.add(nn.Dense(256, activation='relu'))
    # 120 是输出的类别数。
    finetune_net.output_new.add(nn.Dense(120))
    # 初始化输出网络。
    finetune_net.output_new.initialize(init.Xavier(), ctx=ctx)
    # 把模型参数分配到即将用于计算的 CPU 或 GPU 上。
    finetune_net.collect_params().reset_ctx(ctx)
    return finetune_net


In [36]:
net1 = get_net1(ctx)
net2 = get_net2(ctx)
net3 = get_net3(ctx)
net1.load_parameters('resnet50-best.parameters')
net2.load_parameters('resnet101-best.parameters')
net3.load_parameters('resnet152-best-0.17783.parameters')
preds1 = []
preds2 = []
preds3 = []
for data, label in test_data:
    output_features = net1.features(data.as_in_context(ctx))
    output = nd.softmax(net1.output_new(output_features))
    preds1.extend(output.asnumpy())
for data, label in test_data:
    output_features = net2.features(data.as_in_context(ctx))
    output = nd.softmax(net2.output_new(output_features))
    preds2.extend(output.asnumpy())
for data, label in test_data:
    output_features = net3.features(data.as_in_context(ctx))
    output = nd.softmax(net3.output_new(output_features))
    preds3.extend(output.asnumpy())


In [50]:
ids = sorted(os.listdir(os.path.join(data_dir, input_dir, 'test/unknown')))
preds = [(0.18447*j + 0.25398*k) / 2.0 for j,k in zip( preds2, preds3)]
with open('submission-all3-2.csv', 'w') as f:
    f.write('id,' + ','.join(train_valid_ds.synsets) + '\n')
    for i, output in zip(ids, preds):
        f.write(i.split('.')[0] + ',' + ','.join(
            [str(num) for num in output]) + '\n')

In [47]:
0.77252-0.33407, 0.77252-0.25398, 0.77252-0.18447

(0.43845, 0.51854, 0.58805)

In [49]:
0.43845-0.25398, 0.43845-0.18447

(0.18447000000000002, 0.25398)

In [10]:
import os
len(os.listdir('./zjc/test'))

10357